In [ ]:
#split the raw file
raw_file = "twitter-melb.json"
new_doc = "D:\\Jupyter\\COMP90024-Assignment2\\target\\"
file_num = 0
temp = []
int cont = True
with open(raw_file, encoding="UTF-8") as file:
    while cont:
        for i in range(200000):
            new_line = file.readline()
            if not new_line:
                cont = False;
                break;
            temp.append(new_line)
        with open(new_doc+"small_json_file_"+str(file_num)+".json",encoding="UTF-8") as newfile:
            for line in range(i):
                newfile.write(temp[line])
        file_num = file_num + 1
        temp = []

In [1]:
#Spatial.py function
import json
from shapely.geometry import Point, Polygon, MultiPolygon

class SpatialTool:
    regions = {};
    
    def load_region_info(self, filepath):
        with open(filepath, "r") as f:
            for f in json.load(f)["features"]:
                if f["geometry"]["type"] == "Polygon":
                    for polygon in f["geometry"]["coordinates"]:
                        self.regions[f["properties"]["SA2_MAIN16"]] = \
                            Polygon([tuple(p) for p in polygon])
                else:
                    for multi_polygon in f["geometry"]["coordinates"]:
                        self.regions[f["properties"]["SA2_MAIN16"]] = \
                            MultiPolygon([Polygon([tuple(p) for p in polygon]) for polygon in multi_polygon])

    def locate(self, coords):
        """
            Given <tweet["doc"]["coordinates"]["coordinates"]>, 
            return the sa2_code of its corresponding region
            or "outside melbourne"
        """
        for k, v in self.regions.items():
            if (v.contains(Point(coords[0], coords[1]))):
                return k
        return "outside melbourne"

In [6]:
#load the couchdb
import couchdb
import json
import re
from textblob import TextBlob

#change to the real situation
db_server = couchdb.Server("http://127.0.0.1:5984/")
db_server.resource.credentials = ('XXXX', 'XXXX')
#create database in couchdb before running this code
db = db_server["old_twitters"]

In [7]:
tool = SpatialTool()
tool.load_region_info("SA2_2016_MELB.json");

#add json to database
#only add the json in melbourne
for num in range(1,14):
    with open("target/target_list_"+str(num)+".json",encoding='UTF-8') as jsonfile:
        while (True):
            new_line = jsonfile.readline()
            if not new_line:
                break;
            new_json = json.loads(new_line[:-2])
            if (new_json["doc"]["geo"] != None):
                coord_rev = new_json["doc"]["geo"]["coordinates"]
                coord = [coord_rev[1],coord_rev[0]]
                loc_code = tool.locate(coord)
                if (loc_code != "outside melbourne"):
                    new_json.update({"loc_code":loc_code})
                    text = new_json["doc"]["text"]
                    blob = TextBlob(text)
                    value = blob.sentiment.polarity
                    new_json.update({"text_value":value})
                    db.save(new_json)
            elif(new_json["doc"]["place"] != None):
                coord_list = new_json["doc"]["place"]["bounding_box"]["coordinates"][0]
                coord_x = (coord_list[0][0]+coord_list[1][0])/2
                coord_y = (coord_list[0][1]+coord_list[2][1])/2
                coord = [coord_x,coord_y]
                loc_code = tool.locate(coord)
                if (loc_code != "outside melbourne"):
                    new_json.update({"loc_code":loc_code})
                    text = new_json["doc"]["text"]
                    blob = TextBlob(text)
                    value = blob.sentiment.polarity
                    new_json.update({"text_value":value})
                    db.save(new_json)

KeyboardInterrupt: 

In [57]:
#see the view
#create the view in couchdb before running code
for row in db.view('test/new-view',group=True):
    print(row)

<Row key=['206011105', 'ar'], value=134>
<Row key=['206011105', 'bg'], value=1>
<Row key=['206011105', 'da'], value=7>
<Row key=['206011105', 'de'], value=8>
<Row key=['206011105', 'en'], value=4399>
<Row key=['206011105', 'es'], value=36>
<Row key=['206011105', 'et'], value=11>
<Row key=['206011105', 'fi'], value=1>
<Row key=['206011105', 'fr'], value=7>
<Row key=['206011105', 'ht'], value=11>
<Row key=['206011105', 'in'], value=178>
<Row key=['206011105', 'it'], value=5>
<Row key=['206011105', 'ja'], value=36>
<Row key=['206011105', 'ko'], value=1>
<Row key=['206011105', 'lt'], value=1>
<Row key=['206011105', 'lv'], value=1>
<Row key=['206011105', 'nl'], value=7>
<Row key=['206011105', 'pl'], value=10>
<Row key=['206011105', 'pt'], value=4>
<Row key=['206011105', 'ru'], value=2>
<Row key=['206011105', 'sk'], value=2>
<Row key=['206011105', 'sl'], value=7>
<Row key=['206011105', 'sv'], value=3>
<Row key=['206011105', 'th'], value=5>
<Row key=['206011105', 'tl'], value=19>
<Row key=['2

<Row key=['213021341', 'da'], value=2>
<Row key=['213021341', 'de'], value=9>
<Row key=['213021341', 'el'], value=8>
<Row key=['213021341', 'en'], value=1007>
<Row key=['213021341', 'es'], value=8>
<Row key=['213021341', 'et'], value=1>
<Row key=['213021341', 'fr'], value=2>
<Row key=['213021341', 'ht'], value=1>
<Row key=['213021341', 'in'], value=2>
<Row key=['213021341', 'it'], value=3>
<Row key=['213021341', 'ja'], value=1>
<Row key=['213021341', 'ko'], value=2>
<Row key=['213021341', 'nl'], value=4>
<Row key=['213021341', 'no'], value=2>
<Row key=['213021341', 'pt'], value=3>
<Row key=['213021341', 'ru'], value=1>
<Row key=['213021341', 'sv'], value=1>
<Row key=['213021341', 'th'], value=1>
<Row key=['213021341', 'tl'], value=4>
<Row key=['213021341', 'tr'], value=2>
<Row key=['213021341', 'und'], value=42>
<Row key=['213021341', 'vi'], value=1>
<Row key=['213021341', 'zh'], value=1>
<Row key=['213021342', 'en'], value=422>
<Row key=['213021342', 'et'], value=2>
<Row key=['2130213

In [5]:
#output the json from couchdb into a new json file
for x in db:
    with open('twitters.json', 'a+', encoding='utf-8') as f:
        line_json = db[x]
        del line_json["_id"]
        del line_json["_rev"]
        line = json.dumps(line_json, ensure_ascii=False)
        f.write(line+'\n')